## Here we reduce and concatenate the stats

In [1]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def get_fixed_column_names():
    """
    Generate a fixed list of column names to ensure consistency across all files
    """
    columns = []
    
    # Models (excluding model_5)
    models = [f"model_{i}" for i in range(1, 7) if i != 5]
    
    # Metrics for each model
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Statistics to compute
    stats = ['median', 'max', 'min', 'std', 
             'moment1', 'moment2', 'moment3', 'moment4', 'moment5', 
             'q95', 'q90', 'q85', 'q80', 'q20', 'q15', 'q10', 'q05']
    
    # Generate columns for model metrics
    for model in models:
        for metric in metrics:
            for stat in stats:
                columns.append(f"{model}_{metric}_{stat}")
    
    # Generate columns for KL divergence terms
    model_pairs = [(i, j) for i in range(1, 7) for j in range(i+1, 7) 
                  if i != 5 and j != 5]
    
    for i, j in model_pairs:
        for stat in stats:
            columns.append(f"kl_{i}_vs_{j}_{stat}")
    
    return columns

def get_label_from_json(trial_name, question_number, label_folder_path):
    """
    Get label from corresponding JSON file and return 'fact' or 'hallucination'
    based on the 'statement' field
    
    Args:
        trial_name: Name of the trial (e.g., 'NCT001')
        question_number: Question number (1-7)
        label_folder_path: Path to the folder containing JSON files
    
    Returns:
        str: 'fact' if statement is True, 'hallucination' if False, None if error
    """
    if not label_folder_path:  # If no path provided
        return None
        
    json_path = os.path.join(
        label_folder_path,
        f"{trial_name}.json"
    )
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            
            # Get statement value
            statement = data.get('statement')
            
            # Return appropriate label
            if statement == 'True':
                return 'fact'
            elif statement == 'False':
                return 'hallucination'
            else:
                print(f"Warning: Unknown statement value in {trial_name}: {statement}")
                return None
                
    except FileNotFoundError:
        print(f"File not found: {json_path}")
        return None
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {json_path}")
        return None
    except Exception as e:
        print(f"Error reading JSON for {trial_name}: {e}")
        return None

def extract_trial_and_question(file_path):
    """
    Extract trial name and question number from file path
    Example: path/to/NCT00001959/logprob_matrix_1.csv -> ("NCT00001959", 1)
    """
    try:
        # Get filename and remove extension
        filename = os.path.basename(file_path)
        # Extract question number
        question_num = int(filename.split('_')[-1].split('.')[0])
        # Extract trial name from path
        trial_name = file_path.split('/')[-2]  # Adjust this based on your actual path structure
        return trial_name, question_num
    except Exception as e:
        print(f"Error extracting trial and question from {file_path}: {e}")
        return None, None

def compute_statistics(series):
    """
    Compute various statistical measures for a series
    """
    mean = series.mean()
    
    stats = {
        'median': series.median(),
        'max': series.max(),
        'min': series.min(),
        'std': series.std(),
        
        # Central moments
        'moment1': mean,
        'moment2': ((series - mean) ** 2).mean(),
        'moment3': ((series - mean) ** 3).mean(),
        'moment4': ((series - mean) ** 4).mean(),
        'moment5': ((series - mean) ** 5).mean(),
        
        # Existing quantiles
        'q95': series.quantile(0.95),
        'q90': series.quantile(0.90),
        'q85': series.quantile(0.85),
        'q80': series.quantile(0.80),
        'q20': series.quantile(0.20),
        'q15': series.quantile(0.15),
        'q10': series.quantile(0.10),
        'q05': series.quantile(0.05),
    }    
    return stats

def process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path=None):
    """
    Process original metrics and KL divergence terms with consistent column ordering
    """
    # Get fixed column names
    fixed_columns = get_fixed_column_names()
    
    # Original metrics to analyze
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Get all CSV files from model1 directory
    csv_files = []
    model1_path = os.path.join(input_folder, "model_1")
    for root, dirs, files in os.walk(model1_path):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to process")
    print(f"Output will have {len(fixed_columns) + 1} columns")  # +1 for label
    
    for file_path1, file_name in tqdm(csv_files, desc="Processing files"):
        try:
            # Get trial name and question number
            trial_name, question_num = extract_trial_and_question(file_path1)
            
            if trial_name and question_num and label_folder_path:
                # Get label from JSON only if path is provided
                label = get_label_from_json(trial_name, question_num, label_folder_path)
            else:
                label = None
            
            # Dictionary to store all metrics
            all_stats = {col: np.nan for col in fixed_columns}  # Initialize with NaN
            
            # Add label column
            all_stats['label'] = label
            
            # Process each model (excluding model_5)
            models = [f"model_{i}" for i in range(1, 7) if i != 5]
            
            # Step 1: Process original metrics for each model
            for model in models:
                file_path = os.path.join(input_folder, model, os.path.relpath(file_path1, model1_path))
                if not os.path.exists(file_path):
                    print(f"Skipping {file_name} - no matching file in {model}")
                    continue
                
                df = pd.read_csv(file_path)
                
                # Compute statistics for each metric
                for metric in metrics:
                    if metric in df.columns:
                        stats = compute_statistics(df[metric])
                        for stat_name, value in stats.items():
                            col_name = f"{model}_{metric}_{stat_name}"
                            all_stats[col_name] = value
            
            # Step 2: Process KL divergence terms
            kl_file_path = os.path.join(kl_folder, os.path.relpath(file_path1, model1_path))
            if os.path.exists(kl_file_path):
                kl_df = pd.read_csv(kl_file_path)
                
                # Get all KL columns
                kl_cols = [col for col in kl_df.columns if col.startswith('kl_')]
                
                # Compute statistics for each KL term
                for kl_col in kl_cols:
                    stats = compute_statistics(kl_df[kl_col])
                    for stat_name, value in stats.items():
                        col_name = f"{kl_col}_{stat_name}"
                        all_stats[col_name] = value
            
            # Create output DataFrame with fixed column order plus label
            columns_with_label = fixed_columns + ['label']
            result_df = pd.DataFrame([all_stats])[columns_with_label]
            
            # Create output directory structure
            output_file_path = os.path.join(output_folder, os.path.relpath(file_path1, model1_path))
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            
            # Save results
            result_df.to_csv(output_file_path, index=False)
            
        except Exception as e:
            print(f"\nError processing {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue

def concatenate_output_files(output_folder):
    """
    Concatenate all CSV files in Output_folder and its subfolders into a single DataFrame
    """
    # Get list of all CSV files
    csv_files = []
    for root, dirs, files in os.walk(output_folder):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to concatenate")
    # Sort files alphabetically
    csv_files = sorted(csv_files)
    
    # Read and concatenate all files
    all_dfs = []
    for file_path, file_name in tqdm(csv_files, desc="Reading files"):
        try:
            df = pd.read_csv(file_path)
            
            # Add filename as a column (optional but useful for tracking)
            df['source_file'] = file_name
            
            # Add full path as a column (optional)
            df['file_path'] = os.path.relpath(file_path, output_folder)
            
            all_dfs.append(df)
            
        except Exception as e:
            print(f"\nError reading {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue
    
    # Concatenate all DataFrames
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Print some information about the final DataFrame
        print("\nFinal DataFrame info:")
        print(f"Shape: {final_df.shape}")
        print(f"Number of samples: {len(final_df)}")
        print(f"Number of features: {len(final_df.columns)}")
        
        if 'label' in final_df.columns:
            print("\nLabel distribution:")
            print(final_df['label'].value_counts())
        
        return final_df
    else:
        print("No files were successfully read!")
        return None

if __name__ == "__main__":
    # Paths
    input_folder = "Database_free_evaluation/Clinical_trials/Generalization_to_UMLS/files_with_labels"
    kl_folder = "kl_analysis_Paragraph_title_features"
    output_folder = "Output_folder_Paragraph_title"

     # Label folder path (set to None if not using labels)
    label_folder_path = "Database_free_evaluation/Clinical_trials/Generalization_to_UMLS/files_with_labels"
    # Or for other folders:
    # label_folder_path = None
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Process files and create individual outputs
    process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path)

    # Step 2: Concatenate all output files
    final_df = concatenate_output_files(output_folder)
    
    # Optional: Save concatenated DataFrame
    if final_df is not None:
        final_df.to_csv("concatenated_results_Paragraph_title_TESTSET.csv", index=False)

Found 140 CSV files to process
Output will have 681 columns


Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:15<00:00,  8.84it/s]


Found 140 CSV files to concatenate


Reading files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:00<00:00, 151.76it/s]



Final DataFrame info:
Shape: (140, 683)
Number of samples: 140
Number of features: 683

Label distribution:
label
fact             77
hallucination    63
Name: count, dtype: int64


In [3]:
final_df.head(20)

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q90,kl_4_vs_6_q85,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path
0,-0.101243,0.0,-100.000000,7.957377,-1.913819,62.949561,-5558.486186,5.418419e+05,-5.310102e+07,0.0,...,0.338867,0.284243,0.233684,0.000101,0.000022,2.611358e-06,1.495094e-07,fact,logprob_matrix_1.csv,NCT001/logprob_matrix_1.csv
1,-0.184389,0.0,-100.000000,13.725313,-3.043246,186.590078,-17353.034953,1.683384e+06,-1.632044e+08,0.0,...,0.436939,0.388627,0.332166,0.000051,0.000024,3.916730e-06,1.558375e-06,fact,logprob_matrix_2.csv,NCT001/logprob_matrix_2.csv
2,-0.104828,0.0,-10.088431,2.282414,-1.248971,5.184127,-27.636514,2.170131e+02,-1.708334e+03,0.0,...,0.433736,0.359820,0.302641,0.000017,0.000004,1.354503e-06,2.581271e-07,fact,logprob_matrix_3.csv,NCT001/logprob_matrix_3.csv
3,-0.622008,0.0,-100.000000,18.048238,-4.981273,323.522998,-29113.051509,2.772982e+06,-2.634489e+08,0.0,...,0.466100,0.358624,0.290390,0.002143,0.000333,1.018734e-05,2.189004e-06,fact,logprob_matrix_4.csv,NCT001/logprob_matrix_4.csv
4,-0.093424,0.0,-100.000000,11.988374,-2.726284,143.039976,-13106.080152,1.273347e+06,-1.238311e+08,0.0,...,0.451976,0.352293,0.276769,0.000079,0.000005,1.076772e-06,1.921327e-07,fact,logprob_matrix_5.csv,NCT001/logprob_matrix_5.csv
5,-0.091860,0.0,-100.000000,11.153900,-2.566233,123.664509,-11143.041393,1.080548e+06,-1.051829e+08,0.0,...,0.523077,0.436182,0.291353,0.000276,0.000021,2.349377e-06,1.149221e-06,fact,logprob_matrix_6.csv,NCT001/logprob_matrix_6.csv
6,-0.258901,0.0,-100.000000,11.028911,-2.619402,120.904130,-11129.397208,1.083574e+06,-1.055083e+08,0.0,...,0.456004,0.299129,0.229714,0.000275,0.000019,2.907773e-06,2.513672e-07,fact,logprob_matrix_7.csv,NCT001/logprob_matrix_7.csv
7,-0.137985,0.0,-15.018279,2.386667,-1.367059,5.663443,-32.286110,3.124747e+02,-3.400800e+03,0.0,...,0.446325,0.370710,0.308631,0.002054,0.000491,4.433990e-05,2.830322e-06,fact,logprob_matrix_1.csv,NCT004/logprob_matrix_1.csv
8,-0.095941,0.0,-100.000000,15.827896,-3.482843,248.916381,-23034.624587,2.225294e+06,-2.147627e+08,0.0,...,0.396432,0.308901,0.203338,0.002714,0.000513,2.669943e-05,1.320989e-06,fact,logprob_matrix_2.csv,NCT004/logprob_matrix_2.csv
9,-0.258093,0.0,-17.096836,2.584134,-1.460670,6.642414,-48.760314,5.752645e+02,-7.353554e+03,0.0,...,0.389190,0.298376,0.255989,0.003489,0.000887,1.010066e-05,1.504882e-06,fact,logprob_matrix_3.csv,NCT004/logprob_matrix_3.csv


In [7]:
import numpy as np
row = [-6.88, -100, -11.85, -100, -100]
mean= np.mean(row)
mean

-63.746

In [8]:
std = np.std(row)
std

44.4297069988088

In [9]:
(-6.88 - mean) / std

1.2799094083948073

In [6]:
import numpy as np

row = [-6.88, -100, -11.85, -100, -100]  # Fixed the typo in -11,85

# Calculate mean and std of the vector
mean = np.mean(row)
std = np.std(row)

print(f"Mean: {mean}")
print(f"Std: {std}")

# Normalize the first component
normalized_value = (row[0] - mean) / std

print(f"\nOriginal value: {row[0]}")
print(f"Normalized value: {normalized_value}")

Mean: -63.746
Std: 44.4297069988088

Original value: -6.88
Normalized value: 1.2799094083948073
